In [1]:
import os
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc , logfbank
import librosa as lr
import os, glob, pickle
import librosa
from scipy import signal
import noisereduce as nr
from glob import glob
import librosa
get_ipython().magic('matplotlib inline')
import soundfile
from tensorflow.keras.layers import Conv2D,MaxPool2D, Flatten, LSTM
from keras.layers import Dropout,Dense,TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical 
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\HP\AppData\Local\Temp\ipykernel_10548\2007753442.py:16: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('matplotlib inline')


In [2]:
from keras.models import load_model
from keras.layers import Embedding
import pandas as pd
import numpy as np
import re
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gradio as gr

In [45]:
from sklearn.preprocessing import StandardScaler

In [20]:

def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
        # print('yes')
    return result


In [36]:

emotions={
  '1':'neutral',
  '2':'calm',
  '3':'happy',
  '4':'sad',
  '5':'angry',
  '6':'fearful',
  '7':'disgust',
  '8':'surprised'
}


In [37]:
with open("Emotion_Voice_Detection_Model.pkl", 'rb') as file:  
    Emotion_Voice_Detection_Model = pickle.load(file)

In [46]:
def Predict(audio):
    ans  = []
    new_feature = extract_feature(audio, mfcc=True, chroma=True, mel=True)
    ans.append(new_feature)
    ans = np.array(ans)
    a = Emotion_Voice_Detection_Model.predict([ans][0])
    pred = str(a[0])
    return emotions[pred]

In [47]:
iface = gr.Interface(
    fn = Predict,
    inputs=gr.Audio(source="upload", type="filepath"),
    outputs = "text",
    live = True
)

In [48]:
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://31caaf8903ebfc5598.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\gradio\routes.py", line 516, in predict
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\gradio\route_utils.py", line 219, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\gradio\blocks.py", line 1437, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\gradio\blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib(

In [41]:
(extract_feature("Audio.wav", mfcc=True, chroma=True, mel=True))

[-7.26217224e+02  6.85414200e+01  3.29339767e+00  1.22053003e+01
  5.51027775e+00  1.36674080e+01 -2.98382854e+00  3.09802914e+00
 -3.31081295e+00 -1.56438422e+00 -7.86165237e+00 -2.12428164e+00
  2.84920406e+00 -2.66780639e+00  9.59020197e-01  1.62816644e+00
 -2.73668933e+00  2.54241049e-01  2.67537737e+00 -1.76116550e+00
 -1.88647425e+00 -9.75619972e-01 -3.79437268e-01  4.00272876e-01
 -3.04404378e+00 -2.90125823e+00 -1.09248109e-01 -8.63565266e-01
 -3.33326936e+00 -1.97846520e+00  4.57081616e-01 -1.39910948e+00
 -2.92685509e+00  1.39567219e-02 -4.90733743e-01 -5.70905447e-01
  4.03992794e-02 -1.20721745e+00 -1.59498167e+00 -1.43648744e+00
  7.62871325e-01  7.86685467e-01  7.69217014e-01  7.68762350e-01
  7.73604870e-01  7.66614676e-01  7.70436585e-01  7.64893889e-01
  7.80340254e-01  7.61150360e-01  7.59381831e-01  7.33590186e-01
  1.70184157e-05  1.43234653e-03  1.66232847e-02  2.62569860e-02
  1.57336369e-02  1.05670653e-02  2.14492250e-02  2.17550304e-02
  2.05241740e-02  9.86292

In [50]:
pip install pygame

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pygame from https://files.pythonhosted.org/packages/82/61/93ae7afbd931a70510cfdf0a7bb0007540020b8d80bc1d8762ebdc46479b/pygame-2.5.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB 487.6 kB/s eta 0:00:22
   ---------------------------------------- 0.0/10.8 MB 487.6 kB/s eta 0:00:22
   ---------------------------------------- 0.1/10.8 MB 391.3 kB/s eta 0:00:28
   ---------------------------------------- 0.1/10.8 MB 435.7 kB/s eta 0:00:25
   ---------------------------------------- 0.1/10.8 MB 435.7 kB/s eta 0:00:25
   ---------------------------------------- 0.1/10.8 MB 423.5 kB/s eta 0:00:26
    ----------------------------

In [9]:
from playsound import playsound
playsound('DataSet\\Actor_01\\03-01-01-01-01-01-01.wav')

In [44]:
from sklearn.preprocessing import StandardScaler
